In [1]:
import pandas as pd
import json

In [7]:
with open('Vehicular/vehicular_05_08_0000_10_08_0000.json', 'r') as f:
    lines = [json.loads(next(f)) for _ in range(50)]

df = pd.DataFrame(lines)

In [8]:
df.head(50)

,_id,entityId,entityType,acceleration,detectedID,observedBy,heading,confidence,length,insertionTimestamp,location,speed,eventTimestamp,hasDevices,recvTime
0,{'$oid': '66b016005cbff87db0e0b1cd'},urn:ngsi-ld:Bus:112,Bus,161,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:00Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:00Z'},[],{'$date': '2024-08-05T00:00:00Z'}
1,{'$oid': '66b01601970f5810b70c687b'},urn:ngsi-ld:Bus:112,Bus,0,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:01Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:00Z'},[],{'$date': '2024-08-05T00:00:00Z'}
2,{'$oid': '66b016015cbff87db0e0b1d4'},urn:ngsi-ld:Rsu:214,Rsu,0,214,urn:ngsi-ld:Obu:722,3601,100,10,2024-08-05T00:00:01Z,"{'type': 'Point', 'coordinates': [0, 0]}",16383,{'$date': '2024-08-05T00:00:00Z'},[],{'$date': '2024-08-05T00:00:00Z'}
3,{'$oid': '66b01601585e7e683bb88086'},urn:ngsi-ld:Bus:112,Bus,161,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:01Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:01Z'},[],{'$date': '2024-08-05T00:00:01Z'}
4,{'$oid': '66b01602585e7e683bb88088'},urn:ngsi-ld:Bus:112,Bus,161,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:02Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:01Z'},[],{'$date': '2024-08-05T00:00:01Z'}
5,{'$oid': '66b01602e0e6be13bbc2aade'},urn:ngsi-ld:Rsu:214,Rsu,0,214,urn:ngsi-ld:Obu:722,3601,100,10,2024-08-05T00:00:02Z,"{'type': 'Point', 'coordinates': [0, 0]}",16383,{'$date': '2024-08-05T00:00:01Z'},[],{'$date': '2024-08-05T00:00:01Z'}
6,{'$oid': '66b016025cbff87db0e0b1e1'},urn:ngsi-ld:Bus:112,Bus,0,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:02Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:02Z'},[],{'$date': '2024-08-05T00:00:02Z'}
7,{'$oid': '66b016035cbff87db0e0b1e5'},urn:ngsi-ld:Bus:112,Bus,161,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:03Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:02Z'},[],{'$date': '2024-08-05T00:00:02Z'}
8,{'$oid': '66b01603970f5810b70c6881'},urn:ngsi-ld:Rsu:214,Rsu,0,214,urn:ngsi-ld:Obu:722,3601,100,10,2024-08-05T00:00:03Z,"{'type': 'Point', 'coordinates': [0, 0]}",16383,{'$date': '2024-08-05T00:00:02Z'},[],{'$date': '2024-08-05T00:00:02Z'}
9,{'$oid': '66b01603e0e6be13bbc2aae8'},urn:ngsi-ld:Bus:112,Bus,0,112,urn:ngsi-ld:Obu:227,3601,100,10,2024-08-05T00:00:03Z,"{'type': 'Point', 'coordinates': [-8.659536, 4...",0,{'$date': '2024-08-05T00:00:03Z'},[],{'$date': '2024-08-05T00:00:03Z'}


In [2]:
df = pd.read_json('Posts/P1/p1_09_09_1530_14_09_0000.json', lines=True)

In [5]:
df.head(50)

,_id,entityId,entityType,acceleration,detectedID,observedBy,heading,confidence,length,insertionTimestamp,location,speed,eventTimestamp,hasDevices,recvTime
0,{'$oid': '66df1478c56849531152a45e'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:00Z,"{'type': 'Point', 'coordinates': [-8.660236927...",13.6,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
1,{'$oid': '66df1478c56849531152a464'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:00Z,"{'type': 'Point', 'coordinates': [-8.66024227,...",13.6,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
2,{'$oid': '66df1479c56849531152a47c'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660284143...",13.6,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
3,{'$oid': '66df14795eb538763d941071'},urn:ngsi-ld:Car:29337410,Car,-0.144376,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660274056...",13.7,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
4,{'$oid': '66df1479c56849531152a485'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660259281...",13.7,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
5,{'$oid': '66df1479dd447010766e94fe'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660289486...",13.6,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
6,{'$oid': '66df14795eb538763d941079'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660279317...",13.7,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
7,{'$oid': '66df14793077260a3ce8d3bf'},urn:ngsi-ld:Car:29337410,Car,0,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660264624...",13.7,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
8,{'$oid': '66df14793077260a3ce8d3c6'},urn:ngsi-ld:Car:29337410,Car,-0.118746,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660279972...",13.6,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
9,{'$oid': '66df14795eb538763d941087'},urn:ngsi-ld:Car:29337410,Car,0.161979,29337410,urn:ngsi-ld:SlpRadar:001,154.698,99,5.4,2024-09-09T15:30:01Z,"{'type': 'Point', 'coordinates': [-8.660269885...",13.8,{'$date': '2024-09-09T15:30:00Z'},[],{'$date': '2024-09-09T15:30:00Z'}
